# Lab 11: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [238]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2025-04-29 01:17:22,619 | INFO     |: Logger demo_notebook.log already initialized.
2025-04-29 01:17:22,620 | INFO     |: GUI is already running. Shutdown notebook to force restart the GUI.


In [227]:
# Start the plotter
START_PLOTTER()

2025-04-29 01:16:08,207 | ERROR    |: Plotter is already running


# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [244]:
index = 0
sensor_data = [[0.0 for _ in range(1)] for _ in range(18)]
imu_data = [[0.0 for _ in range(1)] for _ in range(18)]
s = None

def notif_handler(uuid, array) :
    global index 
    global sensor_data
    s = ble.bytearray_to_string(array)
    if "*" in s: 
        data = s.split("*")
        if index < 18:
            sensor_data[index] = [float(data[0])]
            imu_data[index] = [float(data[1])]
            index = index + 1 

In [245]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")
            

    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """

        global sensor_data
        global index
        global imu_data

        index = 0 
        sensor_ranges = [[0.0 for _ in range(1)] for _ in range(18)]
        sensor_bearings = [[0.0 for _ in range(1)] for _ in range(18)]

        self.ble.start_notify(self.ble.uuid['RX_STRING'], notif_handler)
        
        # Send command to rotate + scan
        self.ble.send_command(CMD.MAPPING, "")
        
        import asyncio
        asyncio.run(asyncio.sleep(20))
        
        self.ble.stop_notify(self.ble.uuid['RX_STRING'])
        
        sensor_ranges = np.array(sensor_data[1]) / 1000.0
        sensor_bearings = np.array(imu_data[1])[np.newaxis].T

        print(len(sensor_ranges))
        
        return sensor_ranges, sensor_bearings

In [246]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2025-04-29 01:18:05,726 | INFO     |: Already connected to a BLE device


In [247]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2025-04-29 01:18:06,742 | INFO     |:  | Number of observations per grid cell: 18
2025-04-29 01:18:06,743 | INFO     |:  | Precaching Views...
2025-04-29 01:18:07,771 | INFO     |:  | Precaching Time: 1.027 secs
2025-04-29 01:18:07,771 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-29 01:18:07,772 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [248]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)
# cmdr.plot_gt(-0.9144,-0.6096)
cmdr.plot_gt(1.524,-0.9144) # bottom right 
# cmdr.plot_gt(0,0.9144) #Top Middle
# cmdr.plot_gt(1.524,0.9144)

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
# cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])

2025-04-29 01:18:07,990 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-29 01:18:07,990 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
18
2025-04-29 01:18:28,082 | INFO     |: Update Step
2025-04-29 01:18:28,084 | INFO     |:      | Update Time: 0.001 secs
2025-04-29 01:18:28,084 | INFO     |: Bel index     : (np.int64(0), np.int64(0), np.int64(0)) with prob = nan
2025-04-29 01:18:28,085 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-04-29 01:18:28,085 | INFO     |: Belief        : (-1.524, -1.219, -170.000)


Traceback (most recent call last):
  File "/Users/sana/Desktop/FastRobots/FastRobots-sim-release-main/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
                      ^^^^^^^^^^^^^
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
Traceback (most recent call last):
  File "/Users/sana/Desktop/FastRobots/FastRobots-sim-release-main/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
                      ^^^^^^^^^^^^^
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
Traceback (most recent call last):
  File "/Users/sana/Desktop/FastRobots/FastRobots-sim-release-main/src/plotter.py", line 252, in closeEvent
    result = QtGui.QMessageBox.question(self,
             ^^^^^^^^^^^^^^^^^
AttributeError: module 'pyqtgraph.Qt.QtGui' has no attribute 'QMessageBox'


In [6]:
print(" Belief        : (0.914, -1.219, -50.000)")
print(" Belief        : (1.014, -0.910, -50.000)")

 Belief        : (0.914, -1.219, -50.000)
 Belief        : (1.014, -0.910, -50.000)
